In [ ]:
import os
import time

import dill as pickle
from pathlib import Path
import functools
import operator
from datetime import datetime, timedelta
from multiprocessing import Pool
import shutil

import numpy as np
import pandas as pd
pd.options.display.max_rows = 99
pd.options.display.max_columns = 99
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import covid_model_deaths
from covid_model_deaths.data import get_input_data, plot_crude_rates, DeathModelData
from covid_model_deaths.social_distancing_cov import SocialDistCov
from covid_model_deaths.drawer import Drawer
from covid_model_deaths.utilities import CompareModelDeaths, COV_SETTINGS, KS, RATE_THRESHOLD, submit_curvefit, get_peak_date
from covid_model_deaths.impute_death_threshold import impute_death_threshold
from covid_model_deaths.moving_average import moving_average_predictions
from covid_model_deaths.compare_moving_average import CompareAveragingModelDeaths

from db_queries import get_location_metadata

import warnings
warnings.simplefilter('ignore')

ENV = 'prod'
RUN_TYPE = 'prod'
DATESTAMP_LABEL = '2020_04_13_Europe'
DATA_VERSION = 'best'
PEAK_FILE = str(Path(covid_model_deaths.__file__).parent.parent.parent / 'final_peak_locs_04_09')
LSVID = 630

# ensemble plot settings
COLOR_DICT = {
    #'descmax':'dodgerblue',
    #'descmid':'forestgreen',
    'equal':'gold',
    'ascmid':'firebrick',
    'ascmax':'darkviolet'
}
LINE_DICT = {
    #'14':'-',
    '21':'--'
}

CODE_DIR = str(Path(covid_model_deaths.__file__).parent)
OUTPUT_DIR = f'/ihme/covid-19/deaths/{RUN_TYPE}/{DATESTAMP_LABEL}'

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
print(f'Writing to {OUTPUT_DIR}')


In [ ]:
# come up with more informative names...
input_full_df = get_input_data('full_data', DATA_VERSION)
input_death_df = get_input_data('deaths', DATA_VERSION)
input_age_pop_df = get_input_data('age_pop', DATA_VERSION)
input_age_death_df = get_input_data('age_death', DATA_VERSION)

# fix ' names for now
input_full_df['Province/State'] = input_full_df['Province/State'].str.replace("'", "")
input_death_df['Location'] = input_death_df['Location'].str.replace("'", "")

# manually adjust Iceland spike (0 deaths to 5 deaths to 0 deaths in March...)
input_full_df.loc[(input_full_df['Country/Region'] == 'Iceland') & (input_full_df['Date'] == pd.Timestamp('2020-03-15')), ['Deaths', 'Death rate']] = 0
input_death_df = input_death_df.loc[(input_death_df['Country/Region'] != 'Iceland') | (input_death_df['Date'] != pd.Timestamp('2020-03-15'))]
input_death_df.loc[input_death_df['Country/Region'] == 'Iceland', 'Days'] = input_death_df.loc[
    input_death_df['Country/Region'] == 'Iceland', 'Date'
].apply(lambda x: (x - input_death_df.loc[input_death_df['Country/Region'] == 'Iceland', 'Date'].min()).days)

# drop Georgia the country until we fix location_ids
input_full_df = input_full_df[input_full_df['Country/Region'] != 'Georgia'].reset_index(drop=True)
input_death_df = input_death_df[input_death_df['Country/Region'] != 'Georgia'].reset_index(drop=True)

# # make a picture
# plot_crude_rates(input_death_df)


## identify locations we are running
#### European national level outside of Germany, Spain, Italy (the first two being extra-curricular locations)
#### change this to use new location sets

In [ ]:
# get locaton_ids
loc_df = get_location_metadata(location_set_id=111, location_set_version_id=LSVID)
loc_df['location_name'] = loc_df['location_ascii_name']

# Drop any locations in the US and keep only most detailed for modeling
euro_df = loc_df.loc[
    (loc_df['most_detailed']==1) & ~loc_df['path_to_top_parent'].str.startswith('102,'), 
    ['location_id', 'location_name','parent_id','level']
]

# format
euro_df = euro_df.rename(index=str, columns={'location_name':'Location'})
loc_df = loc_df[['location_id', 'location_name']]
loc_df = loc_df.rename(index=str, columns={'location_id':'parent_id',
                                           'location_name':'Country/Region'})
euro_df = euro_df.merge(loc_df)
euro_df['Location'] = euro_df['Location'].str.replace("'", "")


## prepare data for case-to-death

In [ ]:
%%capture

location_ids = sorted(input_full_df.loc[(np.log(input_full_df['Death rate']) > RATE_THRESHOLD), 
                                  'location_id'].unique())

start_time = datetime.now()
def combine_cases_w_backcast_deaths(location_id, input_death_df, input_age_pop_df, input_age_death_df, rate_threshold):
    mod_df = DeathModelData(input_death_df, input_age_pop_df, input_age_death_df, location_id, 'threshold', rate_threshold=RATE_THRESHOLD).df
    mod_df = mod_df.loc[mod_df['location_id'] == location_id].reset_index(drop=True)
    if len(mod_df) > 0:
        date0 = mod_df['Date'].min()
        day0 = mod_df.loc[~mod_df['Date'].isnull(), 'Days'].min()
        mod_df.loc[mod_df['Days'] == 0, 'Date'] = date0 - timedelta(days=np.round(day0))
        mod_df = mod_df.loc[~((mod_df['Deaths'].isnull()) & (mod_df['Date'] == date0))]
        mod_df = mod_df.loc[~mod_df['Date'].isnull()]
        mod_df.loc[mod_df['Death rate'].isnull(), 'Death rate'] = np.exp(mod_df['ln(age-standardized death rate)'])
        mod_df.loc[mod_df['Deaths'].isnull(), 'Deaths'] = mod_df['Death rate'] * mod_df['population']
        mod_df = mod_df.rename(index=str, columns={'Location':'Province/State'})
    else:
        mod_df = pd.DataFrame(
            columns=['location_id', 'Province/State', 'Country/Region', 'Date', 'Deaths', 'Death rate', 'population']
        )

    return mod_df[['location_id', 'Province/State', 'Country/Region', 'Date', 'Deaths', 'Death rate', 'population']].reset_index(drop=True)

_combiner = functools.partial(combine_cases_w_backcast_deaths, 
                              input_death_df=input_death_df, 
                              input_age_pop_df=input_age_pop_df, 
                              input_age_death_df=input_age_death_df, 
                              rate_threshold=RATE_THRESHOLD)
pool = Pool(20)
loc_dfs = pool.map(_combiner, location_ids)
pool.close()
pool.join()
loc_df = pd.concat(loc_dfs)
loc_df = input_full_df[['location_id', 'Province/State', 'Country/Region', 'Date', 'Confirmed', 'Confirmed case rate']].merge(
    loc_df, how='outer'
).reset_index(drop=True)
loc_df.loc[loc_df['Province/State'].isnull(), 'Province/State'] = loc_df['Country/Region']
loc_df['location_id'] = loc_df['location_id'].astype(int)
loc_df.to_csv(f'{OUTPUT_DIR}/backcast_for_case_to_death.csv', index=False)


In [ ]:
loc_df = pd.read_csv(f'{OUTPUT_DIR}/backcast_for_case_to_death.csv')
date_df = impute_death_threshold(loc_df,
                                 location_list=euro_df['Location'].unique().tolist(),
                                 ln_death_rate_threshold=RATE_THRESHOLD)
del loc_df
end_time = datetime.now()
print(end_time - start_time)


## pops

In [ ]:
# store pops for Bobby
pop_df = input_age_pop_df.merge(euro_df).reset_index(drop=True)
pop_df[['location_id', 'Location', 'age_group', 'population']].to_csv(f'{OUTPUT_DIR}/pops.csv', index=False)
print(f'{OUTPUT_DIR}/pops.csv')


## store model data and covariate data, submit models

In [ ]:
submit_all_jobs = False

# method for getting dates
def date_mean(dates):
    dt_min = dates.min()
    deltas = [x-dt_min for x in dates]

    return dt_min + functools.reduce(operator.add, deltas) / len(deltas)

# get mean data from dataset
date_draws = [i for i in date_df.columns if i.startswith('death_date_draw_')]
date_mean_df = date_df.copy()
date_mean_df['threshold_date'] = date_mean_df.apply(
    lambda x: datetime.strptime(date_mean(x[date_draws]).strftime('%Y-%m-%d'), '%Y-%m-%d'),
    axis=1
)
date_mean_df = date_mean_df.rename(index=str, columns={'location':'Location'})
country_label_df = input_full_df[['Province/State', 'Country/Region']].drop_duplicates().reset_index(drop=True)
country_label_df.loc[country_label_df['Province/State'].isnull(), 'Province/State'] = country_label_df['Country/Region']
country_label_df = country_label_df.rename(index=str, columns={'Province/State':'Location'})
date_mean_df = date_mean_df.merge(country_label_df, how='outer')
if date_mean_df['Country/Region'].isnull().any():
    raise ValueError('Trouble attaching country names,')
date_mean_df = date_mean_df[['Location', 'Country/Region', 'threshold_date']]

# set up ensemble
model_out_dirs = []
for cov_sort, weights in COV_SETTINGS:
    for k in KS:
        # set up dirs
        model_out_dir = f'{OUTPUT_DIR}/model_data_{cov_sort}_{k}'
        if not os.path.exists(model_out_dir):
            os.mkdir(model_out_dir)
        model_out_dirs.append(model_out_dir)
        
# how many draws from each scenario
n_scenarios = len(COV_SETTINGS) * len(KS)
n_draws = [int(1000 / n_scenarios)] * n_scenarios
n_draws[-1] = n_draws[-1] + 1000 - np.sum(n_draws)
    
# run models
for location_id, location, country in zip(euro_df['location_id'], euro_df['Location'], euro_df['Country/Region']):
    # run model
    if location == country:
        print(f'Running {location}')
        mod_df = DeathModelData(input_death_df, input_age_pop_df, input_age_death_df, location_id, 'threshold', rate_threshold=RATE_THRESHOLD).df
    else:
        print(f'Running {country} -- {location}')
        mod_df = DeathModelData(input_death_df, input_age_pop_df, input_age_death_df, location_id, 'threshold', subnat=True, rate_threshold=RATE_THRESHOLD).df
    mod_df = mod_df.loc[~(mod_df['Deaths'].isnull())].reset_index(drop=True)
    mod_df = mod_df.loc[~mod_df['Location'].isin(['Life Care Center, Kirkland, WA'])]
    n_i = 0
    for cov_sort, weights in COV_SETTINGS:
        for k in KS:
            # drop back-cast, and submit model
            model_out_dir = f'{OUTPUT_DIR}/model_data_{cov_sort}_{k}'
            if (submit_all_jobs) or (not os.path.exists(f'{model_out_dir}/{location}/draws.pkl')):
                mod_df.loc[mod_df['Location'] == "Valle d'Aosta", 'Location'] = 'Valle dAosta'  # fix back so it matches the file
                mod_df.to_csv(f'{model_out_dir}/{location}.csv', index=False)
                mod_df.loc[mod_df['Location'] == 'Valle dAosta', 'Location'] = "Valle d'Aosta"  # fix so we get the right merge_name (gotta fully move to ids...)
                sd_cov = SocialDistCov(mod_df, date_mean_df)
                sd_cov_df = sd_cov.get_cov_df(weights=weights, k=k)
                sd_cov_df.loc[sd_cov_df['Location'] == "Valle d'Aosta", 'Location'] = 'Valle dAosta'  # fix back so it matches the file
                if cov_sort == 'equal':
                    sd_cov_df.loc[sd_cov_df['Location'].isin(['Denmark', 'Netherlands', 'Norway']), 'cov_1w'] = 1.4
                elif cov_sort == 'ascmid':
                    sd_cov_df.loc[sd_cov_df['Location'].isin(['Denmark', 'Netherlands', 'Norway']), 'cov_1w'] = 1.5
                elif cov_sort == 'ascmax':
                    sd_cov_df.loc[sd_cov_df['Location'].isin(['Denmark', 'Netherlands', 'Norway']), 'cov_1w'] = 1.6
                sd_cov_df.to_csv(f'{model_out_dir}/{location} covariate.csv', index=False)
                if not os.path.exists(f'{model_out_dir}/{location}'):
                    os.mkdir(f'{model_out_dir}/{location}')
                submit_curvefit(job_name=f'curve_model_{location_id}_{cov_sort}_{k}',
                                location_id=location_id, 
                                code_dir=CODE_DIR,
                                env=ENV,
                                model_location=location,
                                model_location_id=location_id,
                                python=shutil.which('python'),
                                data_file=f'{model_out_dir}/{location}.csv', 
                                cov_file=f'{model_out_dir}/{location} covariate.csv', 
                                peaked_file=f'{PEAK_FILE}.csv',
                                output_dir=f'{model_out_dir}/{location}',
                                n_draws=n_draws[n_i])
            n_i = n_i + 1


## compile draws

In [ ]:
# combine everything
draw_dfs = []
past_draw_dfs = []
models_used = []
with PdfPages(f'{OUTPUT_DIR}/ensemble_plot.pdf') as pdf:
    for location_id, location_name in zip(euro_df['location_id'], euro_df['Location']):
        print(location_name)
        data_draws = Drawer(
            ensemble_dirs=model_out_dirs,
            location_name=location_name, 
            location_id=int(location_id), 
            obs_df=input_full_df[(input_full_df['location_id'] == int(location_id))],  #  & (input_full_df['Deaths'] > 0)
            date_draws=date_df.loc[date_df['location'] == location_name, [i for i in date_df.columns if i.startswith('death_date_draw_')]].values, 
            population=input_age_pop_df.loc[input_age_pop_df['location_id'] == int(location_id), 'population'].sum(),
        )
        draw_df, past_draw_df, model_used, days, ensemble_draws = data_draws.get_dated_draws()
        draw_dfs.append(draw_df)
        past_draw_dfs.append(past_draw_df)
        models_used.append(model_used)

        # plot ensemble
        fig, ax = plt.subplots(1, 2, figsize=(11, 8.5))
        for label, draws in ensemble_draws.items():
            label = label.split('model_data_')[1]
            draws = np.exp(draws)
            deaths_mean = draws.mean(axis=0)
            deaths_lower = np.percentile(draws, 2.5, axis=0)
            deaths_upper = np.percentile(draws, 97.5, axis=0)

            d_deaths_mean = (draws[:,1:] - draws[:,:-1]).mean(axis=0)
            d_deaths_lower = np.percentile(draws[:,1:] - draws[:,:-1], 2.5, axis=0)
            d_deaths_upper = np.percentile(draws[:,1:] - draws[:,:-1], 97.5, axis=0)

            # cumulative
            ax[0].fill_between(days,
                               deaths_lower, deaths_upper,
                               color=COLOR_DICT[label.split('_')[0]], 
                               linestyle=LINE_DICT[label.split('_')[1]], 
                               alpha=0.25)
            ax[0].plot(days, deaths_mean, 
                       c=COLOR_DICT[label.split('_')[0]], 
                       linestyle=LINE_DICT[label.split('_')[1]], )
            ax[0].set_title(f'constant: {k}')
            ax[0].set_xlabel('Date')
            ax[0].set_ylabel('Cumulative death rate')

            # daily
            ax[1].fill_between(days[1:],
                               d_deaths_lower, d_deaths_upper,
                               color=COLOR_DICT[label.split('_')[0]], 
                               linestyle=LINE_DICT[label.split('_')[1]], 
                               alpha=0.25)
            ax[1].plot(days[1:], d_deaths_mean, 
                       c=COLOR_DICT[label.split('_')[0]], 
                       linestyle=LINE_DICT[label.split('_')[1]], 
                       label=label.replace('model_data_', ''))
            ax[1].set_xlabel('Date')
            ax[1].set_ylabel('Daily death rates')

        ax[1].legend(loc=2)
        plt.suptitle(f'{location_name} ({model_used})')
        plt.tight_layout()
        pdf.savefig()
if 'location' not in models_used:
    raise ValueError('No location-specific draws used, must be using wrong tag')
draw_df = pd.concat(draw_dfs)
model_type_df = pd.DataFrame({
    'location':euro_df['Location'].to_list(),
    'model_used':models_used
})

# write
# For 4/5/2020 we replicated draw_998 to make the dataframe have 1000 draws
draw_df['draw_999'] = draw_df['draw_998']

draw_df.loc[draw_df['location'] == 'Valle dAosta', 'location'] = "Valle d'Aosta"
draw_df.to_csv(f'{OUTPUT_DIR}/euro_data.csv', index=False)
model_type_df.to_csv(f'{OUTPUT_DIR}/euro_models_used.csv', index=False)
print(f'{OUTPUT_DIR}/euro_data.csv')


## combine with previous predictions

In [ ]:
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##
raw_draw_path = f'{OUTPUT_DIR}/euro_data.csv'
average_draw_path = f'{OUTPUT_DIR}/past_avg_euro_data.csv'
yesterday_draw_path = '/ihme/covid-19/deaths/prod/2020_04_12_Europe/euro_data.csv'
before_yesterday_draw_path = '/ihme/covid-19/deaths/prod/2020_04_11_Europe/euro_data.csv'
## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ## ##

avg_df = moving_average_predictions(
    'Europe', 
    specified=True,
    model_1=raw_draw_path,
    model_2=yesterday_draw_path,
    model_3=before_yesterday_draw_path
)
avg_df['date'] = pd.to_datetime(avg_df['date'])
past_draw_df = pd.concat(past_draw_dfs)

## NO NEED TO DO THIS, FOR NOW ##
# avg_df = get_peak_date(past_draw_df, avg_df)

# store data
avg_df.to_csv(average_draw_path, index=False)
print(average_draw_path)

# plot
plotter = CompareAveragingModelDeaths(
    raw_draw_path=raw_draw_path,
    average_draw_path=average_draw_path,
    yesterday_draw_path=yesterday_draw_path,
    before_yesterday_draw_path=before_yesterday_draw_path
    )
plotter.make_some_pictures(f'{OUTPUT_DIR}/moving_average_compare.pdf',
                            'EEA + Australia + New Zealand')


In [ ]:
plotter = CompareModelDeaths(
    old_draw_path='/ihme/covid-19/deaths/prod/2020_04_12_Europe/past_avg_euro_data.csv',
    new_draw_path=f'{OUTPUT_DIR}/past_avg_euro_data.csv'
)
plotter.make_some_pictures(f'{OUTPUT_DIR}/compare_to_previous.pdf',
                           'EEA + Australia + New Zealand')


In [ ]:
viz_dir = f'/home/j/Project/covid/results/diagnostics/deaths/{DATESTAMP_LABEL}/'
if not os.path.exists(viz_dir):
    os.mkdir(viz_dir)
for viz_fp in ['compare_to_previous.pdf','moving_average_compare.pdf','ensemble_plot.pdf']:
    shutil.copyfile(src=f"{OUTPUT_DIR}/{viz_fp}", dst=f"{viz_dir}/{viz_fp}")
